In [47]:
import pickle
import numpy as np
import pandas as pd
# import seabron as sns
import tensorflow
import sklearn
from tensorflow.keras.models import load_model

In [48]:
## load the trained model and scalling pickle file
import warnings
warnings.filterwarnings('ignore')
model=load_model('classification_model.h5')

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('scaler_x.pkl','rb') as file:
    scaler_x=pickle.load(file)

In [49]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [50]:
input_data=pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [51]:
encoded=label_encoder_geo.transform([input_data['Gender']])
encoded

array([1])

In [52]:
geo_one_encoded=onehot_encoder_geo.transform([input_data['Geography']]).toarray()
geo_one_encoded
geo_ended_df=pd.DataFrame(geo_one_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_ended_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [53]:
input_data=pd.concat([input_data.reset_index(drop=True),geo_ended_df],axis=1)

In [54]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [55]:
input_data['Gender']=label_encoder_geo.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [56]:
input_data.drop(columns=['Geography'],axis=1,inplace=True)

In [57]:
## scalling

In [58]:
X=input_data
print(X)

   CreditScore  Gender  Age  Tenure  Balance  NumOfProducts  HasCrCard  \
0          600       1   40       3    60000              2          1   

   IsActiveMember  EstimatedSalary  Geography_France  Geography_Germany  \
0               1            50000               1.0                0.0   

   Geography_Spain  
0              0.0  


In [65]:
X_scaled=scaler_x.transform(X)

In [66]:
print(X_scaled)

[[-0.53598516  0.91324755  0.10479359 -0.69539349 -0.25781119  0.80843615
   0.64920267  0.97481699 -0.87683221  1.00150113 -0.57946723 -0.57638802]]


In [67]:
# prediction

In [70]:
pred=model.predict(X_scaled)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 938ms/step


array([[0.04601105]], dtype=float32)

In [72]:
prediction_proba=pred[0][0]
prediction_proba

np.float32(0.046011053)

In [73]:
if prediction_proba>0.5:
    print("The customer is likey to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
